# Process Mining с sberpm

Библиотека __`sberpm`__ предназначена для анализа и исследования процессов, построения их в виде графов и решения смежных задач классификации и кластеризации с использованием алгоритмов машинного обучения.

## Оглавление

[DataHolder](#DataHolder)

[Синтетические ID процесса](#Синтетические-ID-процесса) # в полной версии библиотеки

[----------  Традиционный-Process-Mining  ----------](#----------Традиционный-Process-Mining----------)

I. [Майнеры и визуализация графов](#I.-Майнеры-и-визуализация-графов)
 1. [SimpleMiner](#1.-SimpleMiner)
 2. [CausalMiner](#2.-CausalMiner)
 3. [HeuMiner](#3.-HeuMiner)
 4. [AlphaMiner](#4.-AlphaMiner)
 5. [AlphaPlusMiner](#5.-AlphaPlusMiner)
 6. [InductiveMiner](#6.-InductiveMiner)

II. [Метрики](#II.-Метрики)
- [Метрики + графы](#Метрики-+-графы)

III. [Conformance Checking](#III.-Conformance-Checking)

 IV. [BPMN](#IV.-BPMN)

V. [Визуализация](#V.-Визуализация)



[---------- Машинное-обучение ----------](#----------Машинное-обучение----------)

I. [Кластеризация этапов](#I.-Кластеризация-этапов)  # в полной версии библиотеки

II. [Анализ текстов](#II.-Анализ-текстов)  # в полной версии библиотеки

III. [Рекомендательная система](#III.-Рекомендательная-система)  # в полной версии библиотеки

IV. [Сентиментный анализ](#IV.-Сентиментный-анализ) # в полной версии библиотеки

V. [Факторный анализ](#V.-Факторный-анализ) # в полной версии библиотеки

VI. [Поиск аномалий](#VI.-Поиск-аномалий) # в полной версии библиотеки

VII. [Автоинсайты](#VII.-Автоинсайты) # в полной версии библиотеки

VIII. [Имитационное моделирование и what-if анализ](#VIII.-Имитационное-моделирование-и-what-if-анализ)  # в полной версии библиотеки

IX. [Decision Mining](#IX.-Decision-Mining)  # в полной версии библиотеки

X. [Обучение с подкреплением](#X.-Обучение-с-подкреплением)  # в полной версии библиотеки

XI. [Предсказание структуры графа](#XI.-Предсказание-структуры-графа)  # в полной версии библиотеки

XII. [Хронометраж](#XII.-Хронометраж)

XIII. [Поиск счастливого пути](#XIII.-Поиск-счастливого-пути)  # в полной версии библиотеки


# DataHolder

`DataHolder` – это базовый класс для хранения данных. Практически все алгоритмы библиотеки работают с ним (принимают на вход).

Для создания класса `DataHolder` необходимо сперва указать путь к файлу или передать DataFrame конструктору, а затем указать __id_column__ и __activity_column__. Однако, для большинства алгоритмов Process Mining, представленных в библиотеке, этих столбцов недостаточно – необходимы хотя бы одна колонка времени (__start_timestamp_column__ и/или __end_timestamp_column__) и колонка пользователей (__user_column__). 

## Параметры DataHolder
- **data (str or pd.DataFrame)** – путь к файлу данных (.csv, .xls(x), .txt) или pd.DataFrame
- **id_column (str)** – столбец id
- **activity_column (str)** – столбец активностей
- __<font color='red'>*</font>start_timestamp_column (str)__ – время начала активностей
- __<font color='red'>*</font>end_timestamp_column (str)__ – время окончания активностей
- __user_column (str)__ – столбец с именами/id пользователей
- __text_column (str)__ – столбец с текстовыми данными
- __duration_column (str)__ – столбец с длительностями активностей (если не задается, то расчитывается как время_активности_2 - время_активности_1, причем если есть только один столбец со временем, то для последней активности в цепочке ставится NaN)
- __duration_unit (str)__ – размерность (единица измерения) значений в столбце duration_column, если он задан

- __sep (str, default=',')__ – разделительный знак (используется только при чтении данных из файла)
- __encoding (str)__ – кодировка (используется только при чтении данных из файла)
- __nrows (int)__ – количество строк для чтения (используется только при чтении данных из файла)

- __preprocess (bool, default=True)__ – предобработка данных (сортировка, удаление None-значений, преобразование типов)
- __time_format (str)__ – формат временных колонок (обязательно задавать для правильного распознавания даты и ускорения работы). Правила написания: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
- __time_errors: (str, default='raise')__ – действие при ошибке конвертации
- __dayfirst: (bool, default=None)__ – True, если день стоит в начале строки
- __yearfirst: (bool, default=None)__ – True, если год стоит в начале строки
- __n_jobs (int, default=1)__ – максимальное количество потоков, доступное для некоторых вычислений


__<font color='red'>*</font>__ Для большинства алгоритмов нужно задать хотя бы один из временных столбцов. Если нет информации о типе столбца (время начала или конца), следует задать его как __start_timestamp_column__. Для верного распознавания формата также необходимо указать __time_format__.

Исследуемые данные должны представлять собой журнал событий (лог-файл), в котором хранится информация о последовательности (цепочке) событий (активностей) в бизнес-процессах. Пример журнала событий: $W = \{(a,b,c,d), (a,c,b,d), (a,e,d)\}$, где события $a$, $b$, $c$, $d$ и $e$ сортируются по времени.

## Создание DataHolder 
### – с помощью DataFrame

In [ ]:
from sberpm import DataHolder
import pandas as pd

### – с помощью указания пути файла

In [ ]:
path = 'example.xlsx'
data_holder = DataHolder(data=path, 
                         id_column='id', 
                         activity_column='stages', 
                         start_timestamp_column='dt', 
                         user_column='users', 
                         time_format='%Y-%m-%d')

Если данные имеют какой-нибудь разделитель, например '|' как в csv, то после задания колонок, нужно задать параметр __sep='|'__.

## Атрибуты DataHolder
В `DataHolder` названия столбцов хранятся в соответствующих переменных (т.е. нет необходимости запоминать названия колонок):
- id_column
- activity_column
- start_timestamp_column
- end_timestamp_column
- user_column
- text_column
- duration_column

Кроме того, в `DataHolder` хранятся исходные и сгруппированные данные в виде DataFrame, к которым можно обратиться следующим образом:
- data
- grouped_data

## Методы DataHolder
- __check_or_calc_duration__ – рассчитывает длительность каждой активности (в секундах), если это необходимо 
- __get_grouped_data__ – выводит сгруппированные данные по id и указанным колонкам (например, по activity_column и start_timestamp_column)
- __get_unique_activities__ – выводит список уникальных активностей
- __get_columns__ – выводит список с названиями колонок 
- __get_text__ – выводит колонку с текстом, если такая есть
- __get_timestamp_col__ – выводит временную колонку; если их имеется 2, то выводит start_time_column
- __is_interval__ – возвращает True, если это "интервальный лог" (у которого имеются обе временные колонки: начала и конца активности)
- __top_traces_dh__ – возвращает data_holder с данными для n самых частых цепочек

In [ ]:
data_holder.check_or_calc_duration()

In [ ]:
data_holder.data.head()

In [ ]:
data_holder.get_grouped_data(data_holder.activity_column, data_holder.start_timestamp_column).head()

In [ ]:
dh_3 = data_holder.top_traces_dh(3)  # данные только для топ 3 цепочек
dfg = dh_3.get_grouped_data(dh_3.activity_column)
dfg.value_counts(dh_3.activity_column)  # проверка

Имея данные о бизнес-процессе с цепочками статусов и временем начала каждого из них, можно загрузить их в `DataHolder` и построить граф, максимально описывающий этот бизнес-процесс.

# Синтетические ID процесса 

##### <font color='red'>В полной версии библиотеки</font>

__Pro_n_check__ нумерует экземпляры процессов в зависимости от заданных условий. Создает столбец 'pro_n'.

# ----------Традиционный Process Mining----------

## I. Майнеры и визуализация графов

Для построения и отрисовки графа процесса в библиотеке реализовано несколько алгоритмов. Все они хранятся в модуле __`sberpm.miners`__ и имеют один метод:
- __apply__ – строит граф, который сохраняется в поле graph

### 1. SimpleMiner

`SimpleMiner` отрисовывает все ребра, найденные в логе (без какой-либо фильтрации).

В терминах Process Mining:
> Если хотя бы в одной цепочке активностей из лога за некоторой активностью $X$ непосредственно следует активность $Y$ (цепочка вида $...XY...$), то пишут $X>Y$ ($Y$ follows $X$, _follows_ relation).

SimpleMiner рисует ребра между такими парами активностей $X$ и $Y$, если выполняется $X>Y$.

In [ ]:
from sberpm.miners import SimpleMiner

In [ ]:
# Создание объекта SimpleMiner. В конструктор подается DataHolder и параметры алгоритма 
# (у данного майнера параметров нет)
simple_miner = SimpleMiner(data_holder)

# Запуск алгоритма построения графа
simple_miner.apply()

# Сохранение графа
graph = simple_miner.graph

### Визуализация графа
Для визуализации графа следует использовать `GraphvizPainter` из модуля __`sberpm.visual`__

In [ ]:
%matplotlib inline
from sberpm.visual import GraphvizPainter

Класс `GraphvizPainter` имеет методы:
- __apply__ – принимает на вход граф, полученный с помощью майнера, и производит расчет для его отрисовки 
- __write_graph__ – сохраняет граф в требуемом формате (pdf, svg, gv, png)
- __show__ – выводит граф в notebook

In [ ]:
# Создание объекта GraphvizPainter
painter = GraphvizPainter()

# Расчет графа по результатам работы SimpleMiner
painter.apply(graph)

# Можно сохранить граф на жесткий диск в формате png, svg, pdf или gv
painter.write_graph('SimpleMiner.png', format='png')

# Можно вывыести граф в notebook
painter.show()

Класс `Graph` из модуля __`sberpm.visual`__ имеет методы:
- __get_nodes__ – получить все узлы
- __get_edges__ – получить все ребра
- __add_node_metric__ – добавить метрику, связанную с узлами графа
- __add_edge_metric__ – добавить метрику, связанную с ребрами графа
- __clear_node_metrics__ – удалить все метрики с нод (узлов)
- __clear_edge_metrics__ – удалить все метрики с ребер

### 2. CausalMiner

`CausalMiner` основан на фильтрации ребер.
> Производные типы связей от $X>Y$:
- прямые связи ($X \to Y$, _causal_ relation) – это связи, где $Х>Y$ и не $Y>X$
- параллельные связи($X\parallel Y $, _parallel_ relation) – это связи, где $Х>Y$ и $Y>X$
- независимые связи ($X\#Y$, independent) – это связи, где не $X>Y$ и не $Y>X$

CausalMiner рисует ребра между такими парами активностей $X$ и $Y$, если выполняется $X\to Y$.

In [ ]:
from sberpm.miners import CausalMiner

In [ ]:
# Майнер
causal_miner = CausalMiner(data_holder)
causal_miner.apply()
graph = causal_miner.graph

# Отрисовка
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 3. HeuMiner

`HeuMiner` – это эврестический майнер, который удаляет наиболее редкие связи в зависимости от задаваемого порога (threshold). 

Параметр **threshold** принимает значения **от 0 до 1**. Чем он больше, тем меньше ребер на графе (оставшиеся ребра считаются более важными).

Источник: https://www.researchgate.net/publication/229124308_Process_Mining_with_the_Heuristics_Miner-algorithm

In [ ]:
from sberpm.miners import HeuMiner

In [ ]:
# Майнер
heu_miner = HeuMiner(data_holder, threshold=0.8)
heu_miner.apply()
graph = heu_miner.graph

# Отрисовка
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 4. AlphaMiner

`AlphaMiner` рисует граф в виде сетей Петри с учетом прямых, параллельных и независимых связей между активностями. 

In [ ]:
from sberpm.miners import AlphaMiner

In [ ]:
# Майнер
alpha_miner = AlphaMiner(data_holder)
alpha_miner.apply()
graph = alpha_miner.graph

# Отрисовка
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 5. AlphaPlusMiner

`AlphaPlusMiner` – имплементация Alpha+ майнера, который также рисует граф в виде сетей Петри с учетом связей, но в отличие от AlphaMiner может работать с одноцикловыми (one-loop) цепочками вида activity_1$\to$activity_1 (самоцикл).

In [ ]:
from sberpm.miners import AlphaPlusMiner

In [ ]:
# Майнер
alpha_miner_plus = AlphaPlusMiner(data_holder)
alpha_miner_plus.apply()
graph = alpha_miner_plus.graph

# Отрисовка
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 6. InductiveMiner

`InductiveMiner` создаёт дерево процесса. Лисья дерева - реальные активности процесса, остальные вершины - операторы. Есть 4 типа операторов: 
- ПОСЛЕДОВАТЕЛЬНЫЙ (`->`), 
- ИСКЛЮЧАЮЩЕЕ ИЛИ (`X`), 
- ПАРАЛЛЕЛЬНЫЙ (`||`), 
- ЦИКЛ (`*`).

Есть дополнительный 'оператор', который говорит о том, что было невозможно найти ни один из 4 операторов, представленных выше:
- СМЕШЕННАЯ МОДЕЛЬ ('`?`')

*Замечание*: некоторые из листьев дерева могут быть *скрытыми активностями*, отображаемыми чёрными прямоугольниками. Они не являются реальными активностями и используются только для сохранения правильной структуры дерева. 

Например, из лога, состоящего из двух цепочек процесса $W = \{(a, b, c), (a, c)\}$, можно получить следующее дерево процеса:        
`->(a, X(b, скрытая_активность), c)`.

Если во время очередной итерации алгоритм не может найти разрез графа (=подобрать один из 4 операторов), возможно добавить следубщее поведение: если существует активность А, при удалении которой удаётся подобрать оператор, алгоритм возвращает следующее дерево:            
`||(X(активность_А, скрытая_активность), граф_без_активности_А)` - то есть активность А считается параллельной остальному графу.


Это поведение может быть включено или выключено параметром **parallel_activity** в классе `InductiveMiner`.

In [ ]:
from sberpm.miners import InductiveMiner

In [ ]:
# Miner
inductive_miner = InductiveMiner(data_holder)
inductive_miner.apply()
graph = inductive_miner.graph

# Visualization
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

## II. Метрики

На данный момент в модуле __`sberpm.metrics`__ есть 5 основных типов метрик:
1. `ActivityMetric` – метрики по активностям (группировка по activity_column)
2. `TransitionMetric` – метрики по переходам (группировка по уникальным переходам)
3. `IdMetric`– метрики по id (группировка по id_column)
4. `TraceMetric` – метрики по цепочкам активностей (группировка по уникальным цепочкам)
5. `UserMetric` – метрики по пользователям (группировка по user_column)

In [ ]:
from sberpm.metrics import ActivityMetric, TransitionMetric, IdMetric, TraceMetric, UserMetric

Параметры:
- __data_holder__ – объект типа DataHolder, для которого надо рассчитать метрики
- __time_unit__ – единица времени, по умолчанию расчет временных метрик происходит в часах
- __round__ – количество цифр после запятой (только для метрик, значения которых могут быть с плавающей точкой)

Общие методы для всех классов:
- __apply__ – расчет всех характеристик
- __calc_metrics(...)__ – расчет указанных метрик (соответствуют методам/названиям колонок в DataFrame из apply)
- __calculate_time_metrics__ – расчет временных характеристик

- __total_duration__ – расчет суммарного времени работы
- __min_duration__ – расчет минимального времени работы
- __max_duration__ – расчет максимального времени работы
- __mean_duration__ – расчет среднего времени работы для
- __median_duration__ – расчет медианного времени работы
- __std_duration__ – расчет стандартного отклонения времни работы
- __var_duration__ – расчет дисперсии времени работы

Дополнительные методы:
- ActivityMetric
    - __count__ - сколько раз активность встречается в логе
    - __unique_ids__ - уникальные id для каждой активности
    - __unique_ids_num__ - количество уникальных id для каждой активности
    - __aver_count_in_trace__ - среднее количество раз встречаемости активности в цепочке
    - __loop_percent__ - процент зацикленности
    - __throughput__ - частота - количество выполненных активностей за единицу времени
    - __unique_users__ - уникальные пользователи, работавшие с данной активностью
    - __unique_users_num__ - количество уникальных пользователей, работающих над данной активностью
    - __success_rate(...)__ - доля id, имеющих данную активность, которая выполнилась успешно (закончились успешными активностями)
    - __failure_rate(...)__ - доля id, имеющих данную активность, которая выполнилась неуспешно (закончились неуспешными активностями)
    
    
- IdMetric
    - __trace__ - цепочка (список активностей)
    - __trace_length__ - длина цепочки (кол-во активностей в цепочке)
    - __unique_activities__ - уникальные активности в цепочке
    - __unique_activities_num__ - количество уникальных активностей в цепочке
    - __loop_percent__ - процент зацикленности
    - __unique_users__ - уникальные пользователи, работающие с этим ID
    - __unique_users_num__ - кол-во уникальных пользователей, работавших с данным ID

- TraceMetric
    - __count__ - сколько раз данная цепочка встречается в логе
    - __ids__ - уникальные id с данной цепочкой
    - __trace_length__ - длина цепочки (кол-во активностей в цепочке)
    - __unique_activities__ - уникальные активности в цепочке
    - __unique_activities_num__ - количество уникальных активностей в цепочке активностей
    - __unique_users__ - уникальные пользователи, работающие над цепочкой активностей
    - __unique_users_num__ - количество уникальных пользователей, работающих над цепочкой активностей

 
- TransitionMetric
    - __count__ - сколько раз данный переход встречается в логе
    - __unique_ids__ - уникальные id  для каждого перехода
    - __unique_ids_num__ - количество уникальных id с для каждого перехода
    - __aver_count_in_trace__ - среднее количество раз встречаемости объекта в цепочке
    - __loop_percent__ - процент зацикленности
    - __throughput__ - частота - количество выполненных переходов за единицу времени
    - __unique_users__ - уникальные пользователи, работающие над объектом
    - __unique_users_num__ - кол-во уникальных пользователей, работающих над объектом
    - __success_rate(...)__ - доля id, имеющих объект, которые выполнились успешно (закончились успешными активностями)
    - __failure_rate(...)__ - доля id, имеющих объект, которые выполнились неуспешно (закончились неуспешными активностями)
    
    
- UserMetric
    - __count__ - сколько раз данный пользователь встречается в логе
    - __unique_activities__ - уникальные активности, с которыми работал пользователь
    - __unique_activities_num__ - количество уникальных активностей, с которыми работал пользователь
    - __unique_ids__ - уникальные id с данным пользователем
    - __unique_ids_num__ - количество уникальных id с данным пользователем
    - __throughput__ - число раз выполнения объекта за единицу времени
    - __workload__ - доля активности лога, выполненных данным пользователем

### 1. ActivityMetric

In [ ]:
# Создание объекта ActivityMetric
activity_metric = ActivityMetric(data_holder, time_unit='d')

# Расчет всех метрик
activity_metric.apply().head()

### 2. TransitionMetric

In [ ]:
# Создание объекта TransitionMetric
transition_metric = TransitionMetric(data_holder, time_unit='d')

# Расчет всех метрик
transition_metric.apply().head()

### 3. IdMetric

In [ ]:
# Создание объекта IdMetric
id_metric = IdMetric(data_holder, time_unit='d')

# Расчет всех метрик
id_metric.apply().head()

### 4. TraceMetric

In [ ]:
# Создание объекта TraceMetric
trace_metric = TraceMetric(data_holder, time_unit='d')

# Расчет всех метрик
trace_metric.apply().head()

### 5. UserMetric

In [ ]:
# Создание объекта UserMetric
user_metric = UserMetric(data_holder, time_unit='d')

# Расчет всех метрик
user_metric.apply().head()

### Метрики + графы

В библиотеке реализована возможность представить ряд метрик на графе. Сделать это можно в классе `Graph` с помощью методов:
- __add_node_metric__ – добавить метрику, связанную с узлами графа
- __add_edge_metric__ – добавить метрику, связанную с ребрами графа

In [ ]:
# Расчет метрик
nodes_count_metric = activity_metric.count().to_dict()
edges_count_metric = transition_metric.count().to_dict()
mean_time_node_metric = activity_metric.mean_duration().fillna(0).to_dict()

# Получение графа из майнера
graph = causal_miner.graph

# Добавление метрик на граф
graph.add_node_metric('count', nodes_count_metric)
graph.add_edge_metric('count', edges_count_metric)
graph.add_node_metric('mean_time', mean_time_node_metric)

In [ ]:
# Создание объекта GraphvizPainter
painter = GraphvizPainter()

# Отрисовать граф и связать цвет узлов и ребер с нужными метриками
painter.apply(graph, node_style_metric='count', edge_style_metric='count')
# или painter.apply(graph, node_style_metric='mean_time', edge_style_metric='count')

# Сохранение графа
painter.write_graph("metric_graph.png", format = 'png')

# Отображение в jupyter-notebook
painter.show()

Чтобы удалить метрики с графа, следует воспользоваться следующими методами:
- __clear_node_metrics__ – удалить все метрики с нод (узлов)
- __clear_edge_metrics__ – удалить все метрики с ребер

In [ ]:
graph.clear_node_metrics()
graph.clear_edge_metrics()

## III. Conformance Checking

##### <font color='red'>В полной версии библиотеки</font>

`TokenReplay` позволяет рассчитать *fitness*, который показывает, насколько хорошо граф описывает бизнесс-процесс (1 – хорошо, 0 – плохо). Fitness вычисляется отдельно для каждой цепочки (id) при ее проигрывании по сети Петри.

## IV. BPMN

Для сохранения графа в формате BPMN (Business Process Model and Notation) можно воспользоваться `BpmnExporter` из модуля __`sberpm.bpmn`__. Он имеет следующие методы:
- __apply_petri__ – построить BPMN для сети Петри
- __get_string_representation__ – получить BPMN-нотацию графа
- __write__ – записать граф в BPMN формате

На данный момент сохранять можно только графы, полученные из Alpha Miner.

In [ ]:
from sberpm.bpmn import BpmnExporter

In [ ]:
bpmn_exporter = BpmnExporter()
bpmn_exporter.apply(alpha_miner.graph)
bpmn_exporter.get_string_representation()[:1000]

In [ ]:
bpmn_exporter.write('exported.bpmn')

Для загрузки BPMN-файла есть класс `BpmnImporter` со следующими методами:
- __load_bpmn_from_xml__ – загрузить граф, представленный в виде BPMN
- __get_pydotplus_graph__ – получить граф в формате pydotplus

In [ ]:
from sberpm.bpmn import BpmnImporter

In [ ]:
bpmn_importer = BpmnImporter()
bpmn_importer.load_bpmn_from_xml('exported.bpmn')
pydot_graph = bpmn_importer.get_pydotplus_graph()
pydot_graph.write('imported_bpmn.svg', prog='dot', format='svg')

## V. Визуализация

Класс `ChartPainter` из модуля __`sberpm.visual`__ предназначен для создания основных типов графиков. В основе визуализации лежит библиотека __`plotly`__, благодаря чему все диаграммы являются интерактивными. 

In [ ]:
from sberpm.visual import ChartPainter

Параметры:
- __data__ – данные, которые необходимо визуализировать (DataFrame, DataHolder или объекта класса метрик)
- __template__ – стиль графиков, по умолчанию _plotly_
- __palette__ – цветовая палитра графиков, по умолчанию _sequential.Sunset_r_

Каждый метод `ChartPainter` позволяет отрисовать график определенного типа:
- __hist__ – гистограмма
- __bar__ – столбчатая диаграмма
- __box__ – ящичковая диаграмма
- __scatter__ – диаграмма рассеяния
- __line__ – линейный график
- __pie__ – круговая диаграмма
- __sunburst__ – диаграмма солнечные лучи
- __heatmap__ – 2D гистограмма
- __timeline__ – диаграмма Ганта
- __pareto__ – диаграмма Парето

Основные параметры методов (для более подробной информации см. документацию):
- __x__, __y__ – названия столбцов для отрисовки по осям X и Y соответственно
- __sort__ – название столбца для сортировки значений
- __n__ – количество строк для визуализации
- __color__ – название столбца для задания цвета элементам графика
- __subplots__ – кортеж вида (rows, cols, ncols), где rows и cols – это названия столбцов для отрисовки нескольких графиков по рядам и столбцам соответственно, а ncols – это количество столбцов
- __text__ – название столбца с текстовой информацией (или ее вид) для отображения на графике
- __orientation__ – ориентация графика
- __opacity__ – прозрачность элементов графика
- __edge__ – границы элементов графика
- __title__ – название графика

Каждый метод прост в использовании, но при этом обладает достаточно широким функционалом, который позволяет построить графики для любых задач.

### Гистограмма

In [ ]:
painter = ChartPainter(id_metric)
painter.hist(x='total_duration', edge=True)

### Столбчатая диаграмма

In [ ]:
painter = ChartPainter(user_metric)
painter.bar(x=data_holder.user_column, y='total_duration', text=True)

### Диаграмма рассеяния

In [ ]:
painter = ChartPainter(id_metric)
painter.scatter(x='mean_duration', y='median_duration', color='unique_users_num', size='trace_length', 
                edge=True, opacity=0.8)

### Круговая диаграмма

In [ ]:
painter = ChartPainter(user_metric)
painter.pie(labels='count', n=15)

### Гистограмма распрелделения активностей по диапазонам времени

##### По всем активностям

In [ ]:
painter = ChartPainter(data_holder)
painter.hist_activity_of_dur(top= False, use_median=False)

##### По топ активностям

In [ ]:
painter.hist_activity_of_dur(top= True)

##### По одной активности

In [ ]:
painter.hist_activity_of_dur(by_activity='Stage_6',use_median=True)


# ----------Машинное обучение----------

## I. Кластеризация этапов

##### <font color='red'>В полной версии библиотеки</font>

Данный модуль служит для кластеризации этапов процесса, для нахождения близких или идентичных этапов процессов. 

## II. Анализ текстов

##### <font color='red'>В полной версии библиотеки</font>

Данный модуль служит для кластеризации текстов. 
Для каждого кластера алгоритм выдает __номер кластера__ (название кластера или самое близкое сообщение к центроиду кластера), и 10 самых распространенных слов в кластере.

## III. Рекомендательная система

##### <font color='red'>В полной версии библиотеки</font>

Данный модуль представляет собой рекомендательную систему, ранжирующую этапы процесса в порядке их приоритетности для реинжениринга. Система показывает, на какие активности нужно обратить внимание в первую очередь при оптимизации процесса. Сначала выбирается лучшая модель, описывающую зависимость таргетной метрики каждой из Активностей от признаковых метрик всех остальных Активносетй. В качестве метрик могут выступать количество рециклов (количество появлений активности), время выполнения, количество повторений, кастомная метрика пользователя. Далее на основании метрики строятся коэффициенты проблемности, в порядке убывания которых ранжируются Активности.

## IV. Сентиментный анализ

##### <font color='red'>В полной версии библиотеки</font>

Данный модуль представляет собой систему для анализа тональности словесных комментариев в текстовом поле. Модуль анализа тональности имеет два режима: «базовый» и «продвинутый». В «базовом» режиме тональность текста определяется как «positive» или «negative», численное значение тональности определяется в пределах от -1 до 1 (от негатива к позитиву). Границу двух смежных интервалов можно настроить.

## V. Факторный анализ

##### <font color='red'>В полной версии библиотеки</font>

__Факторный анализ__ – это многомерный метод, применяемый для изучения взаимосвязей между значениями переменных. Предполагается, что известные переменные зависят от меньшего количества неизвестных переменных и случайной ошибки. С помощью факторного анализа возможно выявление скрытых переменных факторов, отвечающих за наличие линейных статистических корреляций между наблюдаемыми переменными.

## VI. Поиск аномалий

##### <font color='red'>В полной версии библиотеки</font>

__Выявление аномалий__ (также __обнаружение выбросов__) — это распознавание во время интеллектуального анализа данных редких данных, событий или наблюдений, которые вызывают подозрения ввиду существенного отличия от большей части данных.

##  VII. Автоинсайты

##### <font color='red'>В полной версии библиотеки</font>

Модуль автоинсайтов sberpm.autoinsights позволяет в автоматическом режиме выявить слабые места и уязвимости процесса и наглядно продемонстрировать их на графе процесса. При анализе классом AutoInsights учитываются такие факторы, как:

Длительность этапа
Рост длительности этапа
Не регулярность (редкость) этапа
Зацикленность
Наличие разовых инцидентов влияющие на длительность этапа
Bottle neck
Приводит ли этап приводит к росту времени процесса и/или прочих этапов
В итоге для каждого объекта (активности и ребра) считается метрика от 0 до 1 - "Уровень аномальности". Чем она выше, тем более "инсайтным" является объект.

## VIII. Имитационное моделирование и what-if анализ

##### <font color='red'>В полной версии библиотеки</font>

Модуль имитационного моделирования позволяет симулировать процесс в as-is форме, вносить изменения в процесс и проводить what-if моделирование, а также оценивать качество симуляции по сравнению с исходным лог-файлом 

##  IX. Decision Mining

##### <font color='red'>В полной версии библиотеки</font>

Модуль предназначен для проведения __decision point analysis__, который заключается в определении причин, почему процесс идет по тому или иному пути. Класс `DecisionMining` выявляет, как те или иные свойства (атрибуты) процесса влияют на выбор конкретного пути. 

## X. Обучение с подкреплением

##### <font color='red'>В полной версии библиотеки</font>

В sberpm __Reinforcement Learning (обучение с подкреплением)__ используется для реконструкции процесса в соответствии с заданными критериями.

## XI. Предсказание структуры графа

##### <font color='red'>В полной версии библиотеки</font>

Модель содержит алгоритм предсказания структуры графа, а именно, предсказываются две величины: вероятности и средние времена выполнения нод и рёбер графа. Эти величины представляются как временные ряды, получаемые из имеющихся данных, далее используются ml- и специализированные для работы с временными рядами алгоритмы для предсказания.

## XII. Хронометраж

Рассчёт длительности процесса с предварительной очисткой вывбросов при помоощи алгоритмов машинного обучения.

- __data_holder (SberPM DataHolder)__ - класс с хранящимися данными 
- __start_query__ (str) - запрос указывающий на начало нового процесса
- __end_query__ (str) - запрос указывающий на окончание процесса
- __query__ (str) - запрос указывающий на начало нового процесса или на завершение процесса в данной строке
- __change_columns__ (List[str]) - список с названиями колонок по которым можно определить что начался новый процесс при изменении значения в колонке (например изменения идентификатора процесса или пользователя) 
- __sort_params__ (List[str]) - список названий колонок по которым будет производиться предварительная сортировка данных

Параметры __query__, __start_query__, __end_query__ могут быть типа __"sql"__ или __"pandas"__, они оба должны ссылаться на фрейм данных как __"df"__, они должны возвращать один столбец: булевую маску или столбец из 0 и 1.

В случае, если задаётся __"sql"__ запрос, то он должен выглядеть как __"SELECT ... from df"__.

Метод __get_chrono()__ начнёт процесс рассчёта длительности процесса и в результате выведет словарь(dict) с элементами:
- среднее временя процесса в секундах
- количество отобранных элементов
- количество уникальных процессов
- максимальное количество уникальных идентификаторов рассчитанных в хронометраже 


In [ ]:
from sberpm.ml.chronometrage import Chronometrage

In [ ]:
df = pd.read_excel('chrono_data.xlsx', engine='openpyxl')
dh = DataHolder(df, 'process_id', 'event_type', 'data_timestamp')

In [ ]:
example_start_query = """(df['event_type'] == 'Процесс_16961') & (df['event_action'].isin(['Начало']))"""
example_end_query = """(df['event_type'] == 'Процесс_16961') & (df['event_action'].isin(['Конец']))"""

In [ ]:
cr = Chronometrage(dh, 
                   sort_params=['process_id', 'user_id', 'data_timestamp'], 
                   start_query=example_start_query,
                   end_query=example_end_query,
                   change_columns=['process_id', 'user_id'])
res = cr.get_chrono()

In [ ]:
res

## XIII. Поиск счастливого пути

##### <font color='red'>В полной версии библиотеки</font>

Задача поиска счастливого пути предлагается решается с помощью обучения с подкреплением (RL), которое по своей сути адаптировано к поиску оптимальных действий. RL работает с двумя объектами: агентом и средой. Во время обучения агент взаимодействует с окружающей средой, совершает действия и получает обратную связь, которая называется вознаграждением. Задача формулируется в рамках марковского процесса принятия решений, который основан на:
* множестве состояний в мире 
* множестве действий 
* вероятностном распределении следующего состояния при условии текущего состояния и завершенного действия 
* награды при переходе между состояниями при выполнении действий. 

Выполнение марковского свойства состоит в том, что следующее состояние условно не зависит от прошлых состояний и действий, учитывая текущее состояние и действие. Граф процесса рассматривается как среда, состояния — узлы графа (активности), действия — ребра (выбор следующей активности для перехода), награда — среднее отрицательное время перехода между прошлым и настоящим состояниями. При наличии ключевых состояний за переход в них также начисляется награда. Цель состоит в том, чтобы выбрать оптимальную политику — отображение из пространства состояний в пространство действий или, другими словами, руководство к действию в каждом состоянии — которое максимизирует ожидаемую дисконтированную сумму вознаграждений, проходящих через граф процесса.